In [12]:

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
import seaborn as sns

df = pd.read_csv('data/BTC-USD.csv')
df = df[['Date', 'Close']]
df.rename(columns={'Close': 'value', 'Date': 'date'}, inplace=True)
df = df.set_index('date')
print(df.head())



                 value
date                  
2014-09-17  457.334015
2014-09-18  424.440002
2014-09-19  394.795990
2014-09-20  408.903992
2014-09-21  398.821014


TODO:
1. Create basic model with alpha = 0.1 -- DONE 
2. experiment with different alpha values (.1 vs .4)
3. additive or multiplicative 

alpha - how much importance model will allocate to most recent observation. 

In [8]:
from statsmodels.tsa.holtwinters import ExponentialSmoothing
def triple_exp_smoothing(df, alpha, beta, gamma, n_preds):
  """
    Perform triple exponential smoothing on a pandas dataframe
    using the Holt-Winters method.
    """
  # Split the dataframe into training and testing sets
  train = df.iloc[:-n_preds]
  test = df.iloc[-n_preds:]
  # Fit the model on the training data
  model = ExponentialSmoothing(train, freq='D',
                               trend='add',
                               seasonal='add',
                               seasonal_periods=12).fit(
                                 smoothing_level=alpha,
                                 smoothing_trend=beta,
                                 smoothing_seasonal=gamma)
  # Make predictions on the test data (n_preds-out-of-sample-points)
  preds = model.forecast(n_preds)
  # Concatenate the train and predicted data
  result = pd.concat([train, preds])
  return result

# Specify the smoothing parameters
alpha = 0.1
beta = 0.1
gamma = 0.9
n_preds = 180

result = triple_exp_smoothing(df, alpha, beta, gamma, n_preds)
result.drop(columns='value', inplace=True)
result.rename(columns={0:"value"}, inplace=True)
result.reset_index(inplace=True)
result.dropna(inplace=True)
result.rename(columns={"index":"date"}, inplace=True)
result["date"]= result["date"].dt.date
result.set_index('date')
result



c:\Users\VVV\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning:

After 0.13 initialization must be handled at model creation



,date,value
3003,2022-12-07,16464.904852
3004,2022-12-08,16364.001178
3005,2022-12-09,16347.349460
3006,2022-12-10,16151.274534
3007,2022-12-11,16414.316268
...,...,...
3178,2023-05-31,12986.572316
3179,2023-06-01,12719.573562
3180,2023-06-02,12870.429464
3181,2023-06-03,12691.516323


In [9]:
result_alpha4 = triple_exp_smoothing(df, .4, beta, gamma, n_preds)
result_alpha4.drop(columns='value', inplace=True)
result_alpha4.rename(columns={0:"value"}, inplace=True)
result_alpha4.reset_index(inplace=True)
result_alpha4.dropna(inplace=True)
result_alpha4.rename(columns={"index":"date"}, inplace=True)
#result_alpha4["date"]= result["date"].dt.date
result_alpha4.set_index('date')
result_alpha4

c:\Users\VVV\anaconda3\lib\site-packages\statsmodels\tsa\holtwinters\model.py:427: FutureWarning:

After 0.13 initialization must be handled at model creation



,date,value
3003,2022-12-07 00:00:00,16857.628540
3004,2022-12-08 00:00:00,16795.254034
3005,2022-12-09 00:00:00,17025.578188
3006,2022-12-10 00:00:00,17127.030486
3007,2022-12-11 00:00:00,17715.674619
...,...,...
3178,2023-05-31 00:00:00,27232.678064
3179,2023-06-01 00:00:00,26795.891551
3180,2023-06-02 00:00:00,26474.974717
3181,2023-06-03 00:00:00,25893.658115


In [10]:
# Plot the result
# plt.plot(df.index, df['value'], label='Original')
# plt.plot(result.index, result["value"], label='Triple Exponential Smoothing')
# plt.legend(loc='best')
# plt.show()

In [11]:
#use plotly so we can zoom in
trace1 = go.Scatter(x=df.reset_index()['date'], y=df['value'], mode='lines', name='Actual Price')
trace2 = go.Scatter(x=result.reset_index()['date'], y=result['value'], mode='lines', name='Alpha=.1')
trace3 = go.Scatter(x=result_alpha4.reset_index()['date'], y=result_alpha4['value'], mode='lines', name='Alpha=.4')

data = [trace1, trace2, trace3]
layout = go.Layout(title='Combined Data Plot')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [14]:
#calculate moving averages 
df = pd.read_csv('data/BTC-USD.csv')
df = df[['Date', 'Close']]
df.rename(columns={'Close': 'value', 'Date': 'date'}, inplace=True)
df = df.set_index('date')
print(df.head())

window_size_1 = 50
window_size_2 = 100 

df['Moving_Avg_50'] = df['value'].rolling(window_size_1).mean()
df['Moving_Avg_100'] = df['value'].rolling(window_size_2).mean()
df.tail()

,value,Moving_Avg_50,Moving_Avg_100
date,,,
2023-05-31,27219.658203,28106.404688,26860.073145
2023-06-01,26819.972656,28040.023086,26883.909336
2023-06-02,27249.589844,27977.033555,26914.516797
2023-06-03,27075.128906,27908.822148,26945.793164
2023-06-04,27201.251953,27846.477266,26985.824414


In [15]:
trace1 = go.Scatter(x=df.reset_index()['date'], y=df['value'], mode='lines', name='Actual Price')
trace2 = go.Scatter(x=df.reset_index()['date'],y=df['Moving_Avg_50'], mode='lines', name='MA50')
trace3 = go.Scatter(x=df.reset_index()['date'],y=df['Moving_Avg_100'], mode='lines', name='MA100')

data = [trace1, trace2, trace3]
layout = go.Layout(title='MA Chart - BTC')
fig = go.Figure(data=data, layout=layout)
fig.show()

In [32]:
#calculate level and trends 
df = pd.read_csv('data/BTC-USD.csv')
df = df[['Date', 'Close']]
df.rename(columns={'Close': 'value', 'Date': 'date'}, inplace=True)
df['date'] = pd.to_datetime(df['date'])
df = df.set_index('date')
#print(df.head())

from statsmodels.tsa.seasonal import seasonal_decompose
result = seasonal_decompose(df['value'], model='additive')
print(result.trend)

trend = result.trend
seasonal = result.seasonal

df["trend"] = result.trend 
df["seasonal"]= result.seasonal

trace1 = go.Scatter(x=df.reset_index()['date'], y=df['value'], mode='lines', name='Actual Price')
trace2 = go.Scatter(x=df.reset_index()['date'],y=df['trend'], mode='lines', name='trend')
trace3 = go.Scatter(x=df.reset_index()['date'],y=df['seasonal'], mode='lines', name='seasonal')

data = [trace1, trace2, trace3]
layout = go.Layout(title='Trend Chart - BTC')
fig = go.Figure(data=data, layout=layout)
fig.show()


date
2014-09-17             NaN
2014-09-18             NaN
2014-09-19             NaN
2014-09-20      417.462572
2014-09-21      412.586997
                  ...     
2023-05-31    27414.032924
2023-06-01    27287.690848
2023-06-02             NaN
2023-06-03             NaN
2023-06-04             NaN
Name: trend, Length: 3183, dtype: float64
